# TELCO Analysis

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Cost of acquisition: $315

In [3]:
# We need to pull in the data as a dataframe. To do this, we will use pd.read_csv
telco = pd.read_csv('telco_customer_details.csv')

In [4]:
# Lets look at the top 5 records
telco.head(5)

,customer_id,gender,is_senior_citizen,partner,dependents,phone_service,internet_service,contract_type,payment_type,monthly_charges,total_charges,churn
0,0002-ORFBO,Female,0,Yes,Yes,1,1,1,Mailed check,65.6,593.30,No
1,0003-MKNFE,Male,0,No,No,2,1,0,Mailed check,59.9,542.40,No
2,0004-TLHLJ,Male,0,No,No,1,2,0,Electronic check,73.9,280.85,Yes
3,0011-IGKFF,Male,1,Yes,No,1,2,0,Electronic check,98.0,1237.85,Yes
4,0013-EXCHZ,Female,1,Yes,No,1,2,0,Mailed check,83.9,267.40,Yes


In [5]:
# Based on .info, we can see that there are 11 null values in the total_charges column
telco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   customer_id        7043 non-null   object 
 1   gender             7043 non-null   object 
 2   is_senior_citizen  7043 non-null   int64  
 3   partner            7043 non-null   object 
 4   dependents         7043 non-null   object 
 5   phone_service      7043 non-null   int64  
 6   internet_service   7043 non-null   int64  
 7   contract_type      7043 non-null   int64  
 8   payment_type       7043 non-null   object 
 9   monthly_charges    7043 non-null   float64
 10  total_charges      7032 non-null   float64
 11  churn              7043 non-null   object 
dtypes: float64(2), int64(4), object(6)
memory usage: 660.4+ KB


In [6]:
telco.describe()
# Based on describe, we see the following:
# Because is_senior_citizen is 0 for False and 1 for True, the average gives us the percentage that are True (16.2%)
# phone_service, internet_service, and contract_type are not appropriate to perform these measures of central tendency because they are actually categorical values, not numerical.
# monthly_charges has a mean of $64.76 with a standard deviation of $30.09
# total_charges has a mean of $2283.30 with a standard deviation of $2266.77

,is_senior_citizen,phone_service,internet_service,contract_type,monthly_charges,total_charges
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,1.325004,1.222916,0.690473,64.761692,2283.300441
std,0.368612,0.642730,0.778877,0.833755,30.090047,2266.771362
min,0.000000,0.000000,0.000000,0.000000,18.250000,18.800000
25%,0.000000,1.000000,1.000000,0.000000,35.500000,401.450000
50%,0.000000,1.000000,1.000000,0.000000,70.350000,1397.475000
75%,0.000000,2.000000,2.000000,1.000000,89.850000,3794.737500
max,1.000000,2.000000,2.000000,2.000000,118.750000,8684.800000


In [7]:
# We need to create a column for tenure. We will estimate tenure with total_charges / monthly_charges
telco['tenure'] = round(telco.total_charges / telco.monthly_charges, 0)
telco.head()

,customer_id,gender,is_senior_citizen,partner,dependents,phone_service,internet_service,contract_type,payment_type,monthly_charges,total_charges,churn,tenure
0,0002-ORFBO,Female,0,Yes,Yes,1,1,1,Mailed check,65.6,593.30,No,9.0
1,0003-MKNFE,Male,0,No,No,2,1,0,Mailed check,59.9,542.40,No,9.0
2,0004-TLHLJ,Male,0,No,No,1,2,0,Electronic check,73.9,280.85,Yes,4.0
3,0011-IGKFF,Male,1,Yes,No,1,2,0,Electronic check,98.0,1237.85,Yes,13.0
4,0013-EXCHZ,Female,1,Yes,No,1,2,0,Mailed check,83.9,267.40,Yes,3.0


In [8]:
# A dataframe explaining the values for contract_type in the telco_data dataframe
contract_description_df = pd.DataFrame({"contract_id":[0,1,2], "contract_description": ["Month-to-month", "1 Year", "2 Year"]})
print(contract_description_df)

   contract_id contract_description
0            0       Month-to-month
1            1               1 Year
2            2               2 Year


In [9]:
# A dataframe explaining the values for phone_service in the telco_data dataframe
phone_service_description_df = pd.DataFrame({"phone_service": [0,1,2], "phone_service_description":["No Phone Service", "One Line", "Two or More Lines"]})
print(phone_service_description_df)

   phone_service phone_service_description
0              0          No Phone Service
1              1                  One Line
2              2         Two or More Lines


In [10]:
# A dataframe explaining the values for internet_service in the telco_data dataframe
internet_service_description_df = pd.DataFrame({"internet_service":[0,1,2], "internet_service_description":["No Internet Service", "DSL", "Fiber Optic"]})
print(internet_service_description_df)

   internet_service internet_service_description
0                 0          No Internet Service
1                 1                          DSL
2                 2                  Fiber Optic


In [11]:
# Lets merge the descriptions for the services and contract types with the original dataframe
telco_all = telco.merge(internet_service_description_df).merge(phone_service_description_df)
telco_all = pd.merge(telco_all, contract_description_df, left_on = "contract_type", right_on = "contract_id")
telco_all.drop(columns = 'contract_id', inplace = True)
telco_all.head()

,customer_id,gender,is_senior_citizen,partner,dependents,phone_service,internet_service,contract_type,payment_type,monthly_charges,total_charges,churn,tenure,internet_service_description,phone_service_description,contract_description
0,0002-ORFBO,Female,0,Yes,Yes,1,1,1,Mailed check,65.60,593.30,No,9.0,DSL,One Line,1 Year
1,0022-TCJCI,Male,1,No,No,1,1,1,Credit card (automatic),62.70,2791.50,Yes,45.0,DSL,One Line,1 Year
2,0068-FIGTF,Female,0,No,No,1,1,1,Mailed check,78.20,2078.95,No,27.0,DSL,One Line,1 Year
3,0074-HDKDG,Male,0,Yes,Yes,1,1,1,Bank transfer (automatic),61.60,1611.00,No,26.0,DSL,One Line,1 Year
4,0080-EMYVY,Female,0,No,No,1,1,1,Credit card (automatic),51.45,727.85,No,14.0,DSL,One Line,1 Year


In [12]:
# We are tasked with recommending a plan to reduce churn at Telco. Lets take a look at the overall churn rate across all customers
total_churn_count = telco_all[telco_all.churn == "Yes"].churn.count()
print(f"Number of churned customers: {total_churn_count}")
total_percentage_churned = round(total_churn_count / telco_all.churn.count() * 100, 2)
print(f"Percentage of customers churned: {total_percentage_churned}%")

Number of churned customers: 1869
Percentage of customers churned: 26.54%


In [13]:
telco_all.groupby('churn').customer_id.count()

churn
No     5174
Yes    1869
Name: customer_id, dtype: int64

In [14]:
telco_all.groupby('churn').total_charges.mean()

churn
No     2555.344141
Yes    1531.796094
Name: total_charges, dtype: float64

In [15]:
telco_all.groupby('churn').total_charges.sum()

churn
No     13193241.8
Yes     2862926.9
Name: total_charges, dtype: float64

In [16]:
telco_all.groupby('churn').tenure.mean()

churn
No     37.647298
Yes    17.974853
Name: tenure, dtype: float64

In [17]:
churn = pd.DataFrame(telco_all.groupby(['contract_description','churn'])['customer_id'].count())
churn

customer_id
contract_description churn             
1 Year               No            1307
                     Yes            166
2 Year               No            1647
                     Yes             48
Month-to-month       No            2220
                     Yes           1655

In [18]:
# Lets reset the index to allow for proper analysis
churn.reset_index(inplace = True)
# And rename the customer_id column to count
churn.rename(columns = {'customer_id':'number'}, inplace = True)
churn

,contract_description,churn,number
0,1 Year,No,1307
1,1 Year,Yes,166
2,2 Year,No,1647
3,2 Year,Yes,48
4,Month-to-month,No,2220
5,Month-to-month,Yes,1655


In [19]:
#To run computations horizontally using churn status, lets separate the dataframe and then rejoin it
churn_yes = churn[churn.churn == "Yes"]
churn_yes

,contract_description,churn,number
1,1 Year,Yes,166
3,2 Year,Yes,48
5,Month-to-month,Yes,1655


In [20]:
churn_no = churn[churn.churn == "No"]
churn_no

,contract_description,churn,number
0,1 Year,No,1307
2,2 Year,No,1647
4,Month-to-month,No,2220


In [21]:
churn = pd.merge(churn_yes, churn_no, left_on = "contract_description", right_on = "contract_description", suffixes = ["_yes","_no"])
churn

,contract_description,churn_yes,number_yes,churn_no,number_no
0,1 Year,Yes,166,No,1307
1,2 Year,Yes,48,No,1647
2,Month-to-month,Yes,1655,No,2220


In [22]:
churn.drop(columns = ["churn_yes", "churn_no"], inplace = True)
churn

,contract_description,number_yes,number_no
0,1 Year,166,1307
1,2 Year,48,1647
2,Month-to-month,1655,2220


In [23]:
churn.rename(columns = {"number_yes":"number_churned", "number_no":"number_active"}, inplace = True)
churn

,contract_description,number_churned,number_active
0,1 Year,166,1307
1,2 Year,48,1647
2,Month-to-month,1655,2220


In [24]:
# Lets look at the percentage churned across contract types
churn['percentage_churned'] = round(churn.number_churned / (churn.number_churned + churn.number_active) * 100, 2)
churn

,contract_description,number_churned,number_active,percentage_churned
0,1 Year,166,1307,11.27
1,2 Year,48,1647,2.83
2,Month-to-month,1655,2220,42.71


In [25]:
# Its clear that our month-to-month customers have the greatest churn, so it is this line of business that we will focus our analysis on
# Lets look within our month-to-month group and see if there are specific characteristics associated with churn there
# We will start by creating a dataframe that consists only of our month-to-month customers
telco_mtm = telco_all[telco_all.contract_type == 0]
telco_mtm.head()

,customer_id,gender,is_senior_citizen,partner,dependents,phone_service,internet_service,contract_type,payment_type,monthly_charges,total_charges,churn,tenure,internet_service_description,phone_service_description,contract_description
1473,0013-MHZWF,Female,0,No,Yes,1,1,0,Credit card (automatic),69.40,571.45,No,8.0,DSL,One Line,Month-to-month
1474,0015-UOCOJ,Female,1,No,No,1,1,0,Electronic check,48.20,340.35,No,7.0,DSL,One Line,Month-to-month
1475,0067-DKWBL,Male,1,No,No,1,1,0,Electronic check,49.25,91.10,Yes,2.0,DSL,One Line,Month-to-month
1476,0082-LDZUE,Male,0,No,No,1,1,0,Mailed check,44.30,44.30,No,1.0,DSL,One Line,Month-to-month
1477,0156-FVPTA,Male,0,Yes,No,1,1,0,Electronic check,54.20,1152.70,Yes,21.0,DSL,One Line,Month-to-month


In [26]:
# Lets look at the churn rates based on payment_type
telco_mtm.groupby(['payment_type', 'churn']).customer_id.count()
# While the data shows that customers who pay by electronic check have the highest churn
# I don't have enough domain knowlwedge to know if we can make recommendations here

payment_type               churn
Bank transfer (automatic)  No       388
                           Yes      201
Credit card (automatic)    No       365
                           Yes      178
Electronic check           No       856
                           Yes      994
Mailed check               No       611
                           Yes      282
Name: customer_id, dtype: int64

In [27]:
# Lets look at churn rates based on gender
telco_mtm.groupby(['gender', 'churn']).customer_id.count()
# There doesn't appear to be a significant difference between gender

gender  churn
Female  No       1083
        Yes       842
Male    No       1137
        Yes       813
Name: customer_id, dtype: int64

In [28]:
# When do customers tend to churn anyways...
telco_mtm.groupby(['tenure','churn']).customer_id.count()
# It looks like if people are going to churn, its going to be within the first six months
# But how can we turn that into a recommendation?...

tenure  churn
1.0     No       225
        Yes      382
2.0     No       104
        Yes      120
3.0     No       105
                ... 
70.0    No         3
        Yes        1
71.0    No         5
72.0    No         3
        Yes        2
Name: customer_id, Length: 143, dtype: int64

In [29]:
# What about churn rates based on phone service?
telco_mtm.groupby(['phone_service_description','churn']).customer_id.count()
# It looks like customers with two or more lines are churning the most
# We don't want to recommend that they go to fewer lines

phone_service_description  churn
No Phone Service           No        223
                           Yes       153
One Line                   No       1237
                           Yes       780
Two or More Lines          No        760
                           Yes       722
Name: customer_id, dtype: int64

In [30]:
# What about churn rates based on internet service?
telco_mtm.groupby(['internet_service_description', 'churn']).customer_id.count()
# Well, our fiber optic customers are churning the most, but we can't recommend that they switch to DSL

internet_service_description  churn
DSL                           No        829
                              Yes       394
Fiber Optic                   No        966
                              Yes      1162
No Internet Service           No        425
                              Yes        99
Name: customer_id, dtype: int64

In [31]:
# Just for investigative purposes, what about all combinations of phone service and internet services?
telco_mtm.groupby(['phone_service_description', 'internet_service_description', 'churn']).customer_id.count()
# Combination = Churn Percentage






# One Line/No Internet = 20%
# Two Lines/No Internet = 11%

# No Phone/DSL = 41%
# One Line/DSL = 30%
# Two Lines/DSL = 23%

# One Line/Fiber = 54%
# Two Lines/Fiber = 55%

phone_service_description  internet_service_description  churn
No Phone Service           DSL                           No       223
                                                         Yes      153
One Line                   DSL                           No       438
                                                         Yes      190
                           Fiber Optic                   No       415
                                                         Yes      496
                           No Internet Service           No       384
                                                         Yes       94
Two or More Lines          DSL                           No       168
                                                         Yes       51
                           Fiber Optic                   No       551
                                                         Yes      666
                           No Internet Service           No        41
                           

In [32]:
666/(666+551)

0.5472473294987674

In [33]:
# What about our senior citizens? How do they fare compared to non seniors?
telco_mtm.groupby(['is_senior_citizen','churn']).customer_id.count()
# It looks like our senior citizens are churning at a much higher rate than our non seniors, 
# but they represent a small portion of our book of business. There might be a better target

is_senior_citizen  churn
0                  No       1854
                   Yes      1214
1                  No        366
                   Yes       441
Name: customer_id, dtype: int64

In [34]:
441/(441+366)

0.5464684014869888

In [35]:
# What about customers with partners?
telco_mtm.groupby(['partner','churn']).customer_id.count()
# There is only a 5% difference in churn between customers with partners

partner  churn
No       No       1380
         Yes      1115
Yes      No        840
         Yes       540
Name: customer_id, dtype: int64

In [36]:
# What about customers with dependents?
telco_mtm.groupby(['dependents', 'churn']).customer_id.count()
# Customers without dependents have ~13% higher churn and they represent a large portion of our business
# Perhaps we can market to customers with dependents

dependents  churn
No          No       1690
            Yes      1396
Yes         No        530
            Yes       259
Name: customer_id, dtype: int64

In [37]:
259/(530+259)

0.3282636248415716

In [38]:
# Do we see the same trend in phone lines with customers with dependents?
telco_mtm.groupby(['phone_service_description', 'internet_service_description', 'dependents','churn']).customer_id.count()

phone_service_description  internet_service_description  dependents  churn
No Phone Service           DSL                           No          No       166
                                                                     Yes      133
                                                         Yes         No        57
                                                                     Yes       20
One Line                   DSL                           No          No       323
                                                                     Yes      147
                                                         Yes         No       115
                                                                     Yes       43
                           Fiber Optic                   No          No       339
                                                                     Yes      416
                                                         Yes         No        76
                       

In [39]:
# What about a combination of senior citizens and dependents?
telco_mtm.groupby(['is_senior_citizen', 'dependents', 'churn']).customer_id.count()

is_senior_citizen  dependents  churn
0                  No          No       1355
                               Yes       973
                   Yes         No        499
                               Yes       241
1                  No          No        335
                               Yes       423
                   Yes         No         31
                               Yes        18
Name: customer_id, dtype: int64

In [40]:
55.8-32.6

23.199999999999996

In [41]:
# What about a combination of dependents and partners?
telco_mtm.groupby(['dependents', 'partner', 'churn']).customer_id.count()
# No dependents or partners (46%)
# No dependents yes partner (44%)
# Yes dependents no partner (33%)
# Yes dependents yes partner (33%)

# It appears that the lack of dependents are driving the churn rate relative to partners

dependents  partner  churn
No          No       No       1241
                     Yes      1047
            Yes      No        449
                     Yes       349
Yes         No       No        139
                     Yes        68
            Yes      No        391
                     Yes       191
Name: customer_id, dtype: int64

In [42]:
# What about customers with only a single product line vs customers with both phone and internet?
# Lets add a new column to our telco_mtm dataframe that captures multiple products or not
telco_mtm['has_phone'] = telco_mtm.phone_service.apply(lambda x: 1 if x > 0  else 0)
telco_mtm['has_internet'] = telco_mtm.internet_service.apply(lambda x: 1 if x > 0 else 0)
telco_mtm['has_both'] = telco_mtm.has_phone + telco_mtm.has_internet - 1
telco_mtm.head(50)

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

,customer_id,gender,is_senior_citizen,partner,dependents,phone_service,internet_service,contract_type,payment_type,monthly_charges,total_charges,churn,tenure,internet_service_description,phone_service_description,contract_description,has_phone,has_internet,has_both
1473,0013-MHZWF,Female,0,No,Yes,1,1,0,Credit card (automatic),69.40,571.45,No,8.0,DSL,One Line,Month-to-month,1,1,1
1474,0015-UOCOJ,Female,1,No,No,1,1,0,Electronic check,48.20,340.35,No,7.0,DSL,One Line,Month-to-month,1,1,1
1475,0067-DKWBL,Male,1,No,No,1,1,0,Electronic check,49.25,91.10,Yes,2.0,DSL,One Line,Month-to-month,1,1,1
1476,0082-LDZUE,Male,0,No,No,1,1,0,Mailed check,44.30,44.30,No,1.0,DSL,One Line,Month-to-month,1,1,1
1477,0156-FVPTA,Male,0,Yes,No,1,1,0,Electronic check,54.20,1152.70,Yes,21.0,DSL,One Line,Month-to-month,1,1,1
1478,0191-ZHSKZ,Male,1,No,No,1,1,0,Electronic check,74.75,2111.30,No,28.0,DSL,One Line,Month-to-month,1,1,1
1479,0224-RLWWD,Female,1,No,No,1,1,0,Electronic check,44.30,89.30,No,2.0,DSL,One Line,Month-to-month,1,1,1
1480,0260-ZDLGK,Female,0,No,Yes,1,1,0,Mailed check,72.90,651.40,Yes,9.0,DSL,One Line,Month-to-month,1,1,1
1481,0277-BKSQP,Male,0,Yes,Yes,1,1,0,Electronic check,54.40,475.10,No,9.0,DSL,One Line,Month-to-month,1,1,1
1482,0285-INHLN,Male,0,Yes,Yes,1,1,0,Credit card (automatic),54.75,1510.30,No,28.0,DSL,One Line,Month-to-month,1,1,1


In [43]:
telco_mtm.groupby('has_both').customer_id.count()

has_both
0     900
1    2975
Name: customer_id, dtype: int64

In [44]:
telco_mtm.groupby(['has_both','churn']).customer_id.count()
# Customers that are mtm with only one product have a 28% churn rate
# Customers that are mtm with more than one product line have a 47% churn rate
# We can't recommend that we eliminate product lines for customers to try and keep them :P

has_both  churn
0         No        648
          Yes       252
1         No       1572
          Yes      1403
Name: customer_id, dtype: int64

In [45]:
# So lets consider the possibility that customers who have churned pay a different price than customers who have stayed
telco_mtm.groupby(['churn']).monthly_charges.mean()
# The average active customer pays $61.46 a month
# The average churned customer paid $73.02 a month
# We may be losing customers due to their average payment

churn
No     61.462635
Yes    73.019396
Name: monthly_charges, dtype: float64

In [46]:
# Lets look at what the average active customer has given us vs the average churned customer
telco_mtm.groupby(['churn']).total_charges.mean()
# The average active customer has provided $1521.93 in revenue to the company
# The average churned customer has provided $1164.46 in revenue to the company
# A customer who remains active will provide an average of $357.14 in additional revenue

churn
No     1521.927590
Yes    1164.460574
Name: total_charges, dtype: float64

In [47]:
# We can look even further at how dependents affects total charges
telco_mtm.groupby(['dependents', 'churn']).total_charges.mean()
# In other words, not only do customers with dependents experience 13% less churn
# A churned customer with dependents will provide an additional $180.22 in revenue 
# compared to a churned customer without dependents

dependents  churn
No          No       1519.242308
            Yes      1136.256519
Yes         No       1530.490094
            Yes      1316.479344
Name: total_charges, dtype: float64

In [48]:
# My recommendation is to increase sales and marketing efforts towards families with dependents. 
# Our current cost of aquisition per customer is $315
# We can look at how much revenue a customer with dependents will generate vs how much a customer without dependents will generate
telco_mtm.groupby('dependents').total_charges.mean()

dependents
No     1345.992741
Yes    1460.238150
Name: total_charges, dtype: float64

In [50]:
# Dependents No: $1031 (after subtracting aquisition costs)
# Dependends Yes: $1145 (after subtracting aquisition costs)

telco_all.head()

,customer_id,gender,is_senior_citizen,partner,dependents,phone_service,internet_service,contract_type,payment_type,monthly_charges,total_charges,churn,tenure,internet_service_description,phone_service_description,contract_description
0,0002-ORFBO,Female,0,Yes,Yes,1,1,1,Mailed check,65.60,593.30,No,9.0,DSL,One Line,1 Year
1,0022-TCJCI,Male,1,No,No,1,1,1,Credit card (automatic),62.70,2791.50,Yes,45.0,DSL,One Line,1 Year
2,0068-FIGTF,Female,0,No,No,1,1,1,Mailed check,78.20,2078.95,No,27.0,DSL,One Line,1 Year
3,0074-HDKDG,Male,0,Yes,Yes,1,1,1,Bank transfer (automatic),61.60,1611.00,No,26.0,DSL,One Line,1 Year
4,0080-EMYVY,Female,0,No,No,1,1,1,Credit card (automatic),51.45,727.85,No,14.0,DSL,One Line,1 Year


In [51]:
telco_mtm.head()

,customer_id,gender,is_senior_citizen,partner,dependents,phone_service,internet_service,contract_type,payment_type,monthly_charges,total_charges,churn,tenure,internet_service_description,phone_service_description,contract_description,has_phone,has_internet,has_both
1473,0013-MHZWF,Female,0,No,Yes,1,1,0,Credit card (automatic),69.40,571.45,No,8.0,DSL,One Line,Month-to-month,1,1,1
1474,0015-UOCOJ,Female,1,No,No,1,1,0,Electronic check,48.20,340.35,No,7.0,DSL,One Line,Month-to-month,1,1,1
1475,0067-DKWBL,Male,1,No,No,1,1,0,Electronic check,49.25,91.10,Yes,2.0,DSL,One Line,Month-to-month,1,1,1
1476,0082-LDZUE,Male,0,No,No,1,1,0,Mailed check,44.30,44.30,No,1.0,DSL,One Line,Month-to-month,1,1,1
1477,0156-FVPTA,Male,0,Yes,No,1,1,0,Electronic check,54.20,1152.70,Yes,21.0,DSL,One Line,Month-to-month,1,1,1


In [53]:
telco_mtm.to_csv('telco_mtm.csv', index = False)

In [60]:
telco_mtm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3875 entries, 1473 to 5347
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   customer_id                   3875 non-null   object 
 1   gender                        3875 non-null   object 
 2   is_senior_citizen             3875 non-null   int64  
 3   partner                       3875 non-null   object 
 4   dependents                    3875 non-null   object 
 5   phone_service                 3875 non-null   int64  
 6   internet_service              3875 non-null   int64  
 7   contract_type                 3875 non-null   int64  
 8   payment_type                  3875 non-null   object 
 9   monthly_charges               3875 non-null   float64
 10  total_charges                 3875 non-null   float64
 11  churn                         3875 non-null   object 
 12  tenure                        3875 non-null   float64
 13  